<a href="https://colab.research.google.com/github/Rami-RK/Attention-Transformers/blob/main/Tfr_Decoder_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Decoder Only Transformer (GPT)**

### **Learning Objectives**

At the end of the experiment you will be able to  :

1. understand and implement Causal Self Attention
2. create decoder only architecture i.e. GPT
3. train and generate text from custom built GPT

### **Introduction**

Diagram below is showing the **Decoder only architecture** which is equivalent to the encoder architecture. But the Transformer block contains masked multi-head attention instead of plain multi-head attention.

<br><br>
<center>
<img src= https://www.dropbox.com/scl/fi/2z057e0a8hueqifj5fu9c/decoder_tfr.png?rlkey=3wz68r0h2nxmmlvdxi883qm7t&raw=1 width=950px/>
</center>

Figure below shows the expanded **Transformer block used in Decoder** architecture and we can see the masked multi-head attention. Multi-head attention with masking is known as causal self attention. Causal means model can look only the past.

<br><br>
<center>
<img src= https://www.dropbox.com/scl/fi/9d913xcatk6rxqp9egikl/decoder_tfr_block.png?rlkey=f4f0molugms52qy5ywz7zaxsp&raw=1 width=200px/>
</center>

### **Attention weights for Causal Self-Attention**

* Force attention weights to only pay attention to the past input tokens
* Attention weights are a TxT square matrix
* $ \alpha(i,j) $ is "How much i pays attention to j"
* Causal means  : $ \alpha(i,j) > 0 $ only when $i >= j$
* We can multiply by a mask to enforce this as shown below.
<br><br>
 $ \qquad Initial \ Attention\ weights  \qquad\qquad Helping \ zero \ Upper \ Triangular Matrics \qquad Causal \ Self-Attention $
<br><br>
<center>
<img src= https://www.dropbox.com/scl/fi/zxau8owiop5uwhrw02uly/causal_attention_weigts.png?rlkey=w4rct28y6ntj31ko31s375enz&raw=1 width=950px/>
</center>

### Importing packages

In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import dataset
import numpy as np
import matplotlib.pyplot as plt

### **Implementation of Causal Multihead Attention**

In [ ]:
class CausalSelfAttention(nn.Module):
  def __init__(self, d_k,d_model, n_heads, max_len):
    super().__init__()

    # Assume d_v =d_k
    self.d_k=d_k
    self.n_heads = n_heads

    self.key = nn. Linear (d_model, d_k*n_heads)
    self.query = nn. Linear (d_model, d_k*n_heads)
    self.value = nn. Linear (d_model, d_k*n_heads)

    # final linear layer
    self.fc=nn.Linear (d_k*n_heads, d_model)

    # casual mask
    # make it so that upper diagonal triangle entries are  0.
    # this way we don't have to shift the inputs to make targets

    cm=torch.tril(torch.ones(int(max_len),int(max_len)))
    self.register_buffer("causal_mask",  cm.view(1,1, int(max_len), int(max_len))) # reshaped in 4D
    # register_buffer : to save this tensor once we save the model


  def forward(self, q, k, v, pad_mask=None): # Padding mask --> handles input tokens for padding
    q=self.query(q) # N x T x (hd_k)
    k=self.key(k) # N x T x (hd_k)
    v=self.value(v) # N x T x (hd_k)

    N = q.shape[0]
    T = q.shape[1]


    # change the shape to:
    # (N, T, h, d_k) --> N, h, T, d_k)
    # in order for matrix multiply to work properly
    q=q.view (N, T, self.n_heads, self.d_k).transpose(1,2)
    k=k.view (N, T, self.n_heads, self.d_k).transpose(1,2)
    v=v.view (N, T, self.n_heads, self.d_k).transpose(1,2)

    # Copute attention weights
    # (N, h, T, d_k)   x  (N, h, d_k, T )  --> (N, h, T, T)
    attn_scores = q@k.transpose(-2,-1)/math.sqrt(self.d_k) # Scaled dot product;  @ --> torch.matmul

    if pad_mask is not None:
      attn_scores = attn_scores.masked_fill(pad_mask[:,None,None,:] == 0, float('-inf'))

    attn_scores = attn_scores.masked_fill(self.causal_mask[:, :, :T, :T] == 0, float('-inf'))

    attn_weights = F.softmax(attn_scores, dim =-1)

    # Compute attention-weighted values
    # (N, h, T, T) x (N, h, T, d_k) --> (N, h, T, d_k)
    A = attn_weights @ v

    # reshape it back before final linear layer
    A = A.transpose(1,2)  # (N, T, h, d_k)
    A = A.contiguous(). view(N, T, self.d_k*self.n_heads) # (N, T, h*d_k)

    # projection
    return self.fc(A)

### **Implementation of Transformder Block**

In [ ]:
class TransformerBlock(nn.Module):
  def __init__(self, d_k, d_model, n_heads, max_len, dropout_prob = 0.1):
    super().__init__()

    self.ln1 = nn.LayerNorm(d_model)
    self.ln2 = nn.LayerNorm(d_model)
    self.mha = CausalSelfAttention(d_k, d_model, n_heads, max_len)

    self.ann = nn.Sequential(
        nn.Linear (d_model, d_model*4),
        nn.GELU(),
        nn.Linear(d_model*4, d_model),
        nn.Dropout(dropout_prob),
    )

    self.dropout = nn. Dropout(p=dropout_prob)

  def forward (self, x, pad_mask=None):
    x = self.ln1(x + self.mha(x, x, x, pad_mask))
    x = self.ln2(x + self.ann(x))
    x = self.dropout(x)
    return x

### **Positional Encoding**

In [ ]:
class PositionalEncoding(nn.Module):
  def __init__(self, d_model, max_len =2048, dropout_porb=0.1):
    super().__init__()
    self.dropout = nn.Dropout(p = dropout_porb)

    position = torch.arange(max_len).unsqueeze(1)
    exp_term = torch.arange(0, d_model, 2)
    div_term = torch.exp(exp_term*(-math.log(10000.0)/d_model))
    pe = torch.zeros(1, max_len, d_model)
    pe[0, :, 0::2] = torch.sin(position *div_term)
    pe[0, :, 1::2] = torch.cos(position *div_term)
    self.register_buffer('pe', pe)


  def forward(self, x):
    # x.shape : N x T x D
    x=x + self.pe[:, :x.size(1), :]
    return self.dropout(x)

### **Decoder Transformer**

In [ ]:
class Decoder(nn.Module):
  def __init__(self, vocab_size, max_len, d_k, d_model, n_heads, n_layers, dropout_prob):
    super().__init__()

    self.embedding=nn.Embedding(vocab_size, d_model)

    self.pos_encoding=PositionalEncoding(d_model, max_len, dropout_prob)

    transformer_blocks=[TransformerBlock(d_k, d_model, n_heads, max_len, dropout_prob) for _ in range(n_layers) ]

    self.transformer_blocks = nn.Sequential(*transformer_blocks)

    self.ln = nn.LayerNorm(d_model)
    self.fc = nn.Linear(d_model, vocab_size)

  def forward(self, x, pad_mask=None):
    x=self.embedding(x)
    x=self.pos_encoding(x)
    for block in self.transformer_blocks:
      x = block(x, pad_mask)
    x = self.ln(x)
    x = self.fc(x)  # many-to-many
    return x

### **Testing model forward pass with dummy data**

In [ ]:
model = Decoder(vocab_size = 20_000, max_len=1024, d_k=16, d_model = 64, n_heads = 4,n_layers=2, dropout_prob =0.1)
device =torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # Creating device for GPU and moving model to GPU
print(device)
model.to(device)

cuda:0


Decoder(
  (embedding): Embedding(20000, 64)
  (pos_encoding): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_blocks): Sequential(
    (0): TransformerBlock(
      (ln1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (mha): CausalSelfAttention(
        (key): Linear(in_features=64, out_features=64, bias=True)
        (query): Linear(in_features=64, out_features=64, bias=True)
        (value): Linear(in_features=64, out_features=64, bias=True)
        (fc): Linear(in_features=64, out_features=64, bias=True)
      )
      (ann): Sequential(
        (0): Linear(in_features=64, out_features=256, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=256, out_features=64, bias=True)
        (3): Dropout(p=0.1, inplace=False)
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (ln1): LayerNorm((64,), eps=1e-05,

cuda:0


Decoder(
  (embedding): Embedding(20000, 64)
  (pos_encoding): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_blocks): Sequential(
    (0): TransformerBlock(
      (ln1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (mha): CausalSelfAttention(
        (key): Linear(in_features=64, out_features=64, bias=True)
        (query): Linear(in_features=64, out_features=64, bias=True)
        (value): Linear(in_features=64, out_features=64, bias=True)
        (fc): Linear(in_features=64, out_features=64, bias=True)
      )
      (ann): Sequential(
        (0): Linear(in_features=64, out_features=256, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=256, out_features=64, bias=True)
        (3): Dropout(p=0.1, inplace=False)
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (ln1): LayerNorm((64,), eps=1e-05,

In [ ]:
x = np.random.randint(0,20_000,size=(8,512)) # Batch size of eaight & sequence length of 512, vocab size is 20k
# Token id may be anywhere between 0 to 20_000 exclusive
x_t =torch.tensor(x).to(device)

In [ ]:
y=model(x_t) # without padding
y.shape

512
512


torch.Size([8, 512, 20000])

In [ ]:
mask = np.ones((8,512)) # with padding mask
mask[:, 256:] = 0
mask_t = torch.tensor(mask).to(device)

In [ ]:
y=model(x_t,mask_t) #with mask
y.shape

512
512


torch.Size([8, 512, 20000])

### **Training the Decoder as causal Language Model**

In [ ]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.1 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding

In [ ]:
checkpoint = 'distilbert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
raw_datasets = load_dataset("glue", "sst2") # sst2 is DataSet for sentiment analysis but ignore the label

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [ ]:
def tokenize_fn(batch):
  return tokenizer(batch['sentence'], truncation = True)

In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_fn, batched = True)
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["sentence", "idx", "label"])

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
train_loader =  DataLoader(tokenized_datasets["train"], shuffle = True, batch_size =32, collate_fn = data_collator)
valid_loader = DataLoader(tokenized_datasets["validation"], batch_size = 32, collate_fn = data_collator)

In [ ]:
# Check how it works
for batch in train_loader:
  for k,v in batch.items():
    print("k:", k, "v.shape:", v.shape)
  break

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


k: input_ids v.shape: torch.Size([32, 45])
k: attention_mask v.shape: torch.Size([32, 45])


In [ ]:
tokenizer.pad_token_id

0

In [ ]:
tokenizer.max_model_input_sizes

{'distilbert-base-uncased': 512,
 'distilbert-base-uncased-distilled-squad': 512,
 'distilbert-base-cased': 512,
 'distilbert-base-cased-distilled-squad': 512,
 'distilbert-base-german-cased': 512,
 'distilbert-base-multilingual-cased': 512}

In [ ]:
tokenizer.vocab_size

28996

In [ ]:
model = Decoder (
    vocab_size = tokenizer.vocab_size,
    max_len=tokenizer.max_model_input_sizes[checkpoint],
    d_k=16,
    d_model = 64,
    n_heads = 4,
    n_layers =2,
    dropout_prob =0.1
    )
model.to(device)

Decoder(
  (embedding): Embedding(28996, 64)
  (pos_encoding): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_blocks): Sequential(
    (0): TransformerBlock(
      (ln1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (mha): CausalSelfAttention(
        (key): Linear(in_features=64, out_features=64, bias=True)
        (query): Linear(in_features=64, out_features=64, bias=True)
        (value): Linear(in_features=64, out_features=64, bias=True)
        (fc): Linear(in_features=64, out_features=64, bias=True)
      )
      (ann): Sequential(
        (0): Linear(in_features=64, out_features=256, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=256, out_features=64, bias=True)
        (3): Dropout(p=0.1, inplace=False)
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (ln1): LayerNorm((64,), eps=1e-05,

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss(ignore_index = tokenizer.pad_token_id)
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
from datetime import datetime

In [ ]:
def train(model, criterion, optimzer, train_loader, epochs):
  train_losses = np.zeros(epochs)

  for it in range(epochs):
    model.train()
    t0=datetime.now()
    train_loss = []
    for batch in train_loader:
      # move data to GPU
      batch = {k:v.to(device) for k, v, in batch.items()}

      # zero the parameter gradients
      optimizer.zero_grad()

      # shift the targets backwards
      targets = batch['input_ids'].clone().detach()
      targets = torch.roll(targets,shifts = -1, dims = 1)
      targets[:,-1] = tokenizer.pad_token_id

      # forward pass
      outputs = model(batch['input_ids'], batch['attention_mask'])
      loss = criterion(outputs.transpose(2,1),targets)

      # Backward and optimize
      loss.backward()
      optimizer.step()
      train_loss.append(loss.item())

    # get  train Loss and test loss
    train_loss = np.mean(train_loss)

    # save lossess
    train_losses[it] = train_loss

    dt = datetime.now() -t0
    print(f'Epoch{it+1}/{epochs}, Train Loss: {train_loss:.4f}, Duration {dt}')

  return train_losses

In [ ]:
train_losses = train(model, criterion, optimizer, train_loader, epochs = 4)

Epoch1/4, Train Loss: 5.9568, Duration 0:01:00.053795
Epoch2/4, Train Loss: 5.0103, Duration 0:01:11.757469
Epoch3/4, Train Loss: 4.6799, Duration 0:01:08.021442
Epoch4/4, Train Loss: 4.4923, Duration 0:01:10.056320


In [ ]:
valid_loader = DataLoader(tokenized_datasets["validation"],
                          batch_size =1,
                          collate_fn = data_collator
)


In [ ]:
model.eval()
for batch in valid_loader:
  # move data to GPU
  batch = {k:v.to(device) for k,v in batch.items()}
  outputs = model (batch['input_ids'], batch['attention_mask'])
  break

In [ ]:
outputs.shape

torch.Size([1, 12, 28996])

In [ ]:
torch.argmax(outputs,axis=-1)

tensor([[  170,   112,   188,   170,  1363,   117,   102, 20714,   117,   102,
           102,   117]], device='cuda:0')

In [ ]:
prediction_ids = torch.argmax(outputs, axis=-1)

In [ ]:
tokenizer.decode(prediction_ids[0])

"a's a good, [SEP] awkwardly, [SEP] [SEP],"

In [ ]:
tokenizer.decode(batch['input_ids'][0])

"[CLS] it's a charming and often affecting journey. [SEP]"

In [ ]:
tokenizer.decode(torch.concat((batch['input_ids'][0,:5],prediction_ids[:,4])))

"[CLS] it's a good"

In [ ]:
# Generate something
prompt ="it's "

In [ ]:
tokenized_prompt = tokenizer(prompt, return_tensors='pt')
tokenized_prompt

{'input_ids': tensor([[ 101, 1122,  112,  188,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [ ]:
outputs=model(
    tokenized_prompt['input_ids'][:,:-1].to(device),
    tokenized_prompt['attention_mask'][:,:-1].to(device)
)
outputs.shape

torch.Size([1, 4, 28996])

In [ ]:
prediction_ids = torch.argmax(outputs[:, -1, :], axis=-1)

In [ ]:
tokenizer.decode(prediction_ids[0])

'a'

#### Generating Text

In [ ]:
prompt ="it's a"
tokenized_prompt = tokenizer(prompt, return_tensors='pt')
# prepare inputs + get rid of SEP token at the end
input_ids = tokenized_prompt['input_ids'][:,:-1].to(device)
mask = tokenized_prompt['attention_mask'][:,:-1].to(device)

for _ in range(20):
  outputs = model(input_ids, mask)
  prediction_id = torch.argmax(outputs[:,-1,:], axis = -1)

  input_ids = torch.hstack((input_ids, prediction_id.view(1,1)))
  mask = torch.ones_like(input_ids)

  if prediction_id == tokenizer.sep_token_id:
    break

In [ ]:
tokenizer.decode(input_ids[0])

"[CLS] it's a good time [SEP]"

### **Complete Encoder-Decoder Architecture**

Revisiting again the **Transformer Model Architecture** as per the paper ["**Attention Is All You Need**"](https://arxiv.org/pdf/1706.03762v6.pdf).
<br><br>
<center>
<img src= https://www.dropbox.com/scl/fi/mvnejcz124ibh643kto58/Transformer.png?rlkey=rubq4dknrbnzhptlbn2cx72eq&raw=1 width=750px/>

</center>

We have implemented both Encoder and Decoder seperately. Encoder only architecture is Bert while Decoder only Architecture is GPT. Similary the architecture which contains both Encoder and Decoder is called T5 i.e. `Text to Text Transfer Transformer`

Figure below shows how Encoder and Decoder are connected in full Encoder and Decoder architecture.

<center>
<img src= https://www.dropbox.com/scl/fi/7oa9crzr50diwabd4u41x/encoder_decoder_connection.png?rlkey=u907hop8qljsnbkmbi99940af&raw=1 width=250px/>

</center>
